In [1]:
import numpy as np


def generate_sparse_least_squares(m, n, rho):
    """
    Generates a Sparse Least Squares problem according to the specifications in Nesterov's paper.

    Parameters:
    m (int): Number of rows in matrix A.
    n (int): Number of columns in matrix A, n > m.
    rho (float): Sparsity and magnitude control parameter.

    Returns:
    A (numpy.ndarray): Generated dense matrix A of shape (m, n).
    b (numpy.ndarray): Generated vector b of shape (m,).
    x_star (numpy.ndarray): Sparse solution x* of shape (n,).

    Example:
    m, n, rho = 10, 20, 0.1  # dimensions and sparsity level
    A, b, x_star = generate_sparse_least_squares(m, n, rho)

    # print("Matrix A:\n", A)
    # print("Vector b:\n", b)
    print("Sparse solution x*:\n", x_star)
    """
    # Generate a dense matrix A with elements uniformly distributed in [-1, 1]
    A = np.random.uniform(-1, 1, (m, n))

    # Generate a sparse solution x_star
    x_star = np.zeros(n)
    # Ensure sparsity in the solution
    non_zero_indices = np.random.choice(n, int(n * rho), replace=False)
    x_star[non_zero_indices] = np.random.normal(0, 1, int(n * rho))

    # Calculate the vector b = A*x_star + noise
    # noise = np.random.normal(0, 0.1, m)  # adding small Gaussian noise
    b = np.dot(A, x_star) # + noise

    return A, b, x_star

In [2]:
import numpy as np
import time

class NesterovAcceleratedGradientMethod:
    def __init__(self, A, b, L0, v0, max_iter=10000, tol=1e-6):
        self.A = A
        self.b = b
        self.L = L0
        self.v = v0
        self.max_iter = max_iter
        self.tol = tol
        self.gap_history = []
        self.loss_history = []
        self.cpu_time_history = []

    def gradient_f(self, x):
        return self.A.T @ (self.A @ x - self.b)

    def f(self, x):
        return np.linalg.norm(self.A @ x - self.b)**2 / 2

    def nag_step(self, y, L):
        grad = self.gradient_f(y)
        return y - grad / L

    def compute_steps(self):
        v_prev = self.v
        initial_loss = self.f(self.v)
        
        for k in range(self.max_iter):
            start_time = time.process_time()

            y = self.v + k / (k + 3) * (self.v - v_prev)
            T_L_y = self.nag_step(y, self.L)

            while np.linalg.norm(self.gradient_f(T_L_y) - self.gradient_f(y)) > self.L * np.linalg.norm(T_L_y - y):
                self.L *= 1.1  # Increase L and recompute
                T_L_y = self.nag_step(y, self.L)
            
            v_prev = self.v
            self.v = T_L_y
            
            current_loss = self.f(self.v)
            current_gap = current_loss / initial_loss

            end_time = time.process_time()
            
            self.gap_history.append(current_gap)
            self.loss_history.append(current_loss)
            self.cpu_time_history.append(end_time - start_time)
            
            if current_gap < self.tol:
                break

        return self.v, k, current_gap, self.gap_history, self.loss_history, self.cpu_time_history

# Example usage
m, n, rho = 1000, 4000, 0.1
A, b, x_star = generate_sparse_least_squares(m, n, rho)
v0 = np.zeros(n)  # Initial point
L0 = 1.0          # Initial Lipschitz constant guess

solver = NesterovAcceleratedGradientMethod(A, b, L0, v0, max_iter=1000, tol=1e-9)
solution_nag, iterations_nag, final_gap_nag, gap_history_nag, loss_history_nag, cpu_time_nag = solver.compute_steps()

# Analyzing gap magnitudes and CPU time
def find_magnitude_indices(gap_history, func=np.log10):
    magnitudes = np.floor(func(gap_history))
    unique_magnitudes = []
    indices = []
    for i, magnitude in enumerate(magnitudes):
        if magnitude not in unique_magnitudes:
            unique_magnitudes.append(magnitude)
            indices.append(i + 1)
    return np.array(indices), unique_magnitudes

indices_nag, magnitudes_nag = find_magnitude_indices(gap_history_nag, func=np.log10)
cpu_time_nag = np.array(cpu_time_nag).cumsum()[indices_nag - 1]

# Output results for analysis
print(f"Solution: {solution_nag[:5]}...")  # Showing first few elements of the solution
print(f"Iterations: {iterations_nag}")
print(f"Final Gap: {final_gap_nag}")
print(f"Indices of significant gap changes: {indices_nag}")
print(f"CPU time at significant changes: {cpu_time_nag}")


Solution: [-0.01874395  0.0236633   0.0107372  -0.02279081  0.07950713]...
Iterations: 46
Final Gap: 6.405778562169117e-10
Indices of significant gap changes: [ 1  2  4  6 10 14 21 29 37 47]
CPU time at significant changes: [1.953125 2.03125  2.03125  2.046875 2.09375  2.125    2.15625  2.203125
 2.328125 2.515625]


In [3]:
import numpy as np
import time

class DualGradientMethod:
    def __init__(self, A, b, penalty, gamma_u, gamma_d, L_0, v_0, max_iter=10000, tol=1e-6) -> None:
        self.A = A
        self.b = b
        self.penalty = penalty
        self.gamma_u = gamma_u
        self.gamma_d = gamma_d
        self.L = L_0
        self.v = v_0
        self.max_iter = max_iter
        self.tol = tol
        self.gap_history = []
        self.loss_history = []
        self.cpu_time_history = []

    def gradient_f(self, x):
        return 2 * np.dot(self.A.T, (np.dot(self.A, x) - self.b))

    def f(self, x):
        return np.linalg.norm(np.dot(self.A, x) - self.b)**2 / 2

    def psi(self, x):
        # Placeholder for potential regularizer function
        return 0

    def compute_steps(self):
        v_prev = self.v
        initial_residual = np.dot(self.A, self.v) - self.b
        initial_loss = np.linalg.norm(initial_residual)**2 / 2
        
        for k in range(self.max_iter):
            start_time = time.process_time()

            # Nesterov acceleration
            if k > 0:
                y = self.v + (k - 1) / (k + 2) * (self.v - v_prev)
            else:
                y = self.v

            grad_f = self.gradient_f(y)
            Mk = np.linalg.norm(grad_f, 2)
            if Mk == 0:
                Mk = 1e-16  # Prevent division by zero

            L_k = max(self.L, Mk / self.gamma_d)
            T, L_new = self.gradient_iteration(self.penalty, self.gamma_u, y, L_k)
            self.L = L_new
            
            v_prev = self.v
            self.v = T
            
            current_residual = np.dot(self.A, self.v) - self.b
            current_loss = np.linalg.norm(current_residual)**2 / 2
            current_gap = current_loss / initial_loss

            end_time = time.process_time()

            self.gap_history.append(current_gap)
            self.loss_history.append(current_loss)
            self.cpu_time_history.append(end_time - start_time)

            if current_gap < self.tol:
                break

        return self.v, k, current_gap, self.gap_history, self.loss_history, self.cpu_time_history

    def gradient_iteration(self, penalty, gamma_u, x, M):
        L = M
        while True:
            changes = self.gradient_f(x) - L * x
            T = three_cases(changes, penalty, L)
            if not self.psi(T) > np.dot(self.gradient_f(T), (x - T)) + np.linalg.norm(x - T)**2 * L / 2 + self.psi(x):
                break
            L *= gamma_u
        return T, L

def three_cases(changes, penalty, denominator):
    # Vectorized handling of three cases for the proximal gradient update
    return (np.less_equal(changes, -penalty) * (-changes - penalty) + np.greater_equal(changes, penalty) * (-changes + penalty)) / denominator



In [4]:
# Parameters
m, n = 1000, 4000  # dimensions
rho = 0.1          # sparsity factor
tol = 1e-9         # tolerance for relative gap reduction

# Generate the problem
A, b, x_star = generate_sparse_least_squares(m, n, rho)

# Instantiate and solve using DualGradientMethod
v0 = np.zeros(n)
L0 = 2
gamma_d = 2
penalty = 0.5  # This replaces lambda_ which was likely a regularization parameter

solver = DualGradientMethod(A, b, penalty, gamma_d=2, gamma_u=1.1, L_0=L0, v_0=v0, max_iter=10000, tol=tol)
solution_dual, iterations_dual, final_gap_dual, gap_history_dual, loss_history_dual, cpu_history_dual = solver.compute_steps()

# Analyzing gap magnitudes and CPU time
def find_magnitude_indices(gap_history, func=np.log10):
    magnitudes = np.floor(func(gap_history))
    unique_magnitudes = []
    indices = []
    for i, magnitude in enumerate(magnitudes):
        if magnitude not in unique_magnitudes:
            unique_magnitudes.append(magnitude)
            indices.append(i + 1)
    indices = np.array(indices)
    return indices, unique_magnitudes

indices_dual, magnitudes_dual = find_magnitude_indices(gap_history_dual, func=np.log2)
cpu_history_dual = np.array(cpu_history_dual).cumsum()[indices_dual - 1]  # Adjusting indices for Python's 0-based index

# Output results for analysis
print(f"Solution: {solution_dual[:5]}...")  # Showing first few elements of the solution
print(f"Iterations: {iterations_dual}")
print(f"Final Gap: {final_gap_dual}")
print(f"Indices of significant gap changes: {indices_dual}")
print(f"CPU time at significant changes: {cpu_history_dual}")


Solution: [-2.80889804e-06 -1.14807672e-07  2.08181017e-06  4.13181627e-07
 -2.71063769e-01]...
Iterations: 9999
Final Gap: 0.00010730729782251724
Indices of significant gap changes: [   1    2    3    5    6    7    8    9   11   12   13   16   20   23
   26   29   36   39   43   61 1079]
CPU time at significant changes: [ 0.        0.046875  0.046875  0.046875  0.0625    0.078125  0.078125
  0.09375   0.09375   0.09375   0.140625  0.203125  0.265625  0.265625
  0.3125    0.375     0.40625   0.46875   0.484375  0.71875  14.6875  ]


In [5]:
class PrimalGradientMethod:
    def __init__(self, A, b, x0, tol, max_iter=10000):
        self.A = A
        self.b = b
        self.x = x0
        self.tol = tol
        self.max_iter = max_iter
        self.gap_history = []
        self.loss_history = []
        self.cpu_time_history = []

    def gradient_f(self, x):
        return 2 * np.dot(self.A.T, (np.dot(self.A, x) - self.b))

    def f(self, x):
        return np.linalg.norm(np.dot(self.A, x) - self.b)**2

    def compute_steps(self):
        initial_loss = self.f(self.x)
        self.loss_history.append(initial_loss)
        
        for k in range(self.max_iter):
            start_time = time.process_time()
            gradient = self.gradient_f(self.x)
            step_size = 0.1 / (np.linalg.norm(gradient) if np.linalg.norm(gradient) != 0 else 1e-16)
            self.x -= step_size * gradient
            current_loss = self.f(self.x)
            relative_gap = current_loss / initial_loss

            self.loss_history.append(current_loss)
            self.gap_history.append(relative_gap)
            end_time = time.process_time()
            self.cpu_time_history.append(end_time - start_time)

            if relative_gap < self.tol:
                break

        return self.x, k, relative_gap, self.gap_history, self.loss_history, self.cpu_time_history

# Parameters
m, n = 1000, 4000  # dimensions
rho = 0.1          # sparsity factor
tol = 1e-9         # tolerance for relative gap reduction

# Generate the problem
A, b, x_star = generate_sparse_least_squares(m, n, rho)
x0 = np.zeros(n)

# Instantiate and solve using PrimalGradientMethod
solver = PrimalGradientMethod(A, b, x0, tol, max_iter=10000)
solution, iterations, final_gap, gap_history, loss_history, cpu_time = solver.compute_steps()

# Analyzing gap magnitudes and CPU time
def find_magnitude_indices(gap_history, func=np.log2):
    magnitudes = np.floor(func(gap_history))
    unique_magnitudes = []
    indices = []
    for i, magnitude in enumerate(magnitudes):
        if magnitude not in unique_magnitudes:
            unique_magnitudes.append(magnitude)
            indices.append(i + 1)
    indices = np.array(indices)
    return indices, unique_magnitudes

indices, magnitudes = find_magnitude_indices(gap_history)
cpu_time = np.array(cpu_time).cumsum()[indices - 1]  # Adjusting indices for Python's 0-based index

# Output results for analysis
print(f"Solution: {solution[:5]}...")  # Showing first few elements of the solution
print(f"Iterations: {iterations}")
print(f"Final Gap: {final_gap}")
print(f"Indices of significant gap changes: {indices}")
print(f"CPU time at significant changes: {cpu_time}")


Solution: [-0.11493664  0.22963825  0.14850047  0.23265202  0.07101003]...
Iterations: 9999
Final Gap: 7.286066231851823e-05
Indices of significant gap changes: [  1  28  48  64  76  85  92  97 101 104 106 108 109 110 111 112 113 115
 120]
CPU time at significant changes: [0.       0.109375 0.1875   0.203125 0.25     0.265625 0.40625  0.4375
 0.453125 0.453125 0.46875  0.46875  0.46875  0.515625 0.515625 0.515625
 0.515625 0.53125  0.5625  ]
